In [62]:
import os
from dotenv import load_dotenv
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')

# Define your connection string
username = 'postgres'
password = 'a7xforever'
host = 'localhost'  # or your actual host
port = '5432'  # or your actual port if different
database = 'chinook'

connection_string = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}'
db = SQLDatabase.from_uri(connection_string)

# print(db.get_usable_table_names())


llm = ChatOpenAI(api_key=openai_api_key, temperature=0, model="gpt-4o-mini")
chain = create_sql_query_chain(llm, db)

chain.get_prompts()[0].pretty_print()

You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURRENT_DATE function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLR

In [66]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm,db)

# chain = write_query | execute_query
# chain.invoke({"question": "monthly total sales over all the years."})

print(write_query.invoke({"question": "How many employees are there"}))

```sql
SELECT COUNT("employee_id") AS employee_count FROM employee;
```


In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import